In [7]:
import numpy as np
import time
import cupy as cp

In [8]:
cross = cp.ElementwiseKernel(
     'raw float64 a,raw float64 b, float64 c', '',
     '''
     int z=i%3;
     if (z==0){
     c=a[i+1]*b[i+2]-a[i+2]*b[i+1];
     }
     else if (z==1){
     c=a[i+1]*b[i-1]-a[i-1]*b[i+1];
     }
     else{
     c=a[i-2]*b[i-1]-a[i-1]*b[i-2];
     }
     ''', 'my_kernel')

In [9]:
L=900
s=cp.zeros((L,L,3))
n_iter=1001 #10001
K=0
imp_prob=0 #impurity probability
R=cp.random.choice(a=[True, False], size=(L, L), p=[imp_prob, 1-imp_prob]) #impurtiy lattice matrix

f=open('corr.csv','ab')

In [10]:
white = cp.ones((L, L),dtype='float32')

kernel = cp.ElementwiseKernel(
     'float32 x, int16 L', 'float32 z',
     '''
     int b=i%L;
     int a=(i-b)/L;
     if ((a+b)%2 == 0) {
       z = 1;
     } 
     else {
       z = 0;
     }''', 'my_kernel')

white=kernel(white,L)
black=1-white

In [11]:
def monte(s,theta,phi,B,T,K,colour):
    J=1
    D=6**0.5
    beta=1/T
    e=0.2#1
    
    del_theta=e*(cp.random.rand(L,L)-0.5)
    del_phi=e*(cp.random.rand(L,L)-0.5)
    theta2=(theta+del_theta)%(np.pi) #theta has to be in range (0,pi)
    phi2=(phi+del_phi)%(2*np.pi)
    
    s2=cp.zeros((L,L,3))
    s2[:,:,0]=cp.sin(theta2)*cp.cos(phi2)
    s2[:,:,1]=cp.sin(theta2)*cp.sin(phi2)
    s2[:,:,2]=cp.cos(theta2)
    y=cp.array([0,1,0])
    x=cp.array([1,0,0])
    
    sx=cp.roll(s,-1,axis=0)
    sx_=cp.roll(s,1,axis=0)
    sy=cp.roll(s,-1,axis=1)
    sy_=cp.roll(s,1,axis=1)
    
    E_J=-J*cp.dot(cp.multiply(s2-s,sx+sx_+sy+sy_),cp.ones(3))
    E_B=-B*(s2-s)[:,:,2]
    t1=cp.zeros((L,L,3))
    cross(s2-s,sx-sx_,t1)
    t2=cp.zeros((L,L,3))
    cross(s2-s,sy-sy_,t2)
    E_D=+D*cp.dot(t1,y) - D*cp.dot(t2,x)
    E_T=-T*(cp.log(cp.sin(theta2)+0.00001)-cp.log(cp.sin(theta)+0.00001))
    E_A=-K*R*(s2[:,:,2]*s2[:,:,2]-s[:,:,2]*s[:,:,2])
    del_E=E_J+E_B+E_D+E_T+E_A
    
    trans_prob= cp.logical_or(del_E<0, cp.exp(-beta*del_E)>cp.random.rand(L,L))
    if colour==1:
        trans_prob=cp.multiply(trans_prob,white)
    else:
        trans_prob=cp.multiply(trans_prob,black)

    theta=(theta+cp.multiply(trans_prob,del_theta))%(np.pi) #theta has to be in range (0,pi)
    phi=(phi+cp.multiply(trans_prob,del_phi))%(2*np.pi)
    p=cp.zeros((L,L,3))
    p[:,:,0]=s[:,:,0]+cp.multiply(trans_prob,(s2-s)[:,:,0])
    p[:,:,1]=s[:,:,1]+cp.multiply(trans_prob,(s2-s)[:,:,1])
    p[:,:,2]=s[:,:,2]+cp.multiply(trans_prob,(s2-s)[:,:,2])
    
    return [p,theta,phi]

In [12]:
for B in [4]:
    for n in range(1):
        #print(n,end=' ')
        theta=cp.random.rand(L,L)*(np.pi)
        phi=cp.random.rand(L,L)*(2*np.pi)
        
        for T in range(101):
            cp.zeros((L,L,3))
            temp=2*np.exp(-0.04*T)
            t = time.time()
            for m in range(n_iter):
                if T>95 and m==1000:
                    s_temp=cp.asnumpy(cp.reshape(s,(1,-1)))
                    np.savetxt(f,s_temp,delimiter=',')
                [s,theta,phi]=monte(s,theta,phi,B,temp,K,0)
                [s,theta,phi]=monte(s,theta,phi,B,temp,K,1)
            print("{0:.1f}".format(time.time() - t),end=' ')
            
    print('\n',"Magnetic Field: ",B)
    print("\n")
f.close()

41.7 41.9 41.9 42.2 42.3 42.3 42.3 42.6 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 42.7 45.3 45.1 45.1 45.1 45.1 
 Magnetic Field:  4


